In [328]:
#import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


In [329]:
#load data
train_data = pd.read_csv('train.csv')

In [330]:
train_data.head()

,PatientID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,761,52,M,ASY,112,230,0,Normal,160,N,0.0,Up,1
1,181,51,M,ATA,130,224,0,Normal,150,N,0.0,Up,0
2,309,57,M,ASY,95,0,1,Normal,182,N,0.7,Down,1
3,84,56,M,ASY,150,213,1,Normal,125,Y,1.0,Flat,1
4,88,43,M,TA,120,291,0,ST,155,N,0.0,Flat,1


In [331]:
test_data = pd.read_csv('test_X.csv')

In [332]:
#extract features
X = train_data.drop(['PatientID', 'HeartDisease'], axis=1)
X_target = test_data.drop('PatientID', axis=1)

# extract labels
y = train_data['HeartDisease']

In [333]:
#replace the 0 in cholestral to the mean value of the rest, how come when you do this the F1 decrease? after trying ways I decide to delete the row that has missing data
#non_zero_mean = X_train[X_train['Cholesterol'] != 0]['Cholesterol'].mean()

#X_train['Cholesterol'] = X_train['Cholesterol'].replace(0, non_zero_mean)
X_train= X_train[X_train['Cholesterol'] != 0]

In [334]:
#split the data into training and test sets (80%training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [335]:
# Encode categorical features (e.g., Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope)
categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
encoder = LabelEncoder()
for feature in categorical_features:
    X_train[feature] = encoder.fit_transform(X_train[feature])
    X_test[feature] = encoder.transform(X_test[feature])

In [336]:
#feature selection
sel = VarianceThreshold(.8 * (1 - .8))
sel.fit_transform(X_train)

array([[59. ,  1. ,  0. , ...,  1. ,  1.2,  1. ],
       [63. ,  1. ,  0. , ...,  0. ,  1.4,  1. ],
       [53. ,  0. ,  2. , ...,  0. ,  0. ,  2. ],
       ...,
       [62. ,  1. ,  2. , ...,  1. ,  1.2,  1. ],
       [59. ,  1. ,  3. , ...,  0. ,  0. ,  2. ],
       [37. ,  1. ,  2. , ...,  0. ,  3.5,  0. ]])

In [337]:
#standarize numerical features
numerical_features = ['Age', 'RestingBP', 'FastingBS','Cholesterol', 'MaxHR', 'Oldpeak']
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

In [338]:
# Choose a classifier (you can try different classifiers)
#hdp = LogisticRegression()
# Create an SVM classifier instance (example with a linear kernel)
#svm_classifier = svm.SVC(kernel='linear', C=1.0)
# Train the SVM classifier on the training data
#svm_classifier.fit(X_train, y_train)
# Create a Random Forest classifier instance
random_forest_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the Random Forest classifier on the training data
random_forest_classifier.fit(X_train, y_train)

# Train the classifier
#hdp.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [339]:
# make prediction
#y_pred = hdp.predict(X_test)

# Make predictions on the test data
#y_pred = svm_classifier.predict(X_test)
# Make predictions on the test data
y_pred = random_forest_classifier.predict(X_test)

In [340]:
# evaluataion
print ('F1 =' ,f1_score(y_test, y_pred))

F1 = 0.8620689655172413
